## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/train (1).csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(20800, 5)

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
###Drop Nan Values
df=df.dropna()


In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:
## Get the Dependent features
y=df['label']

In [12]:
X.shape

(18285, 4)

In [13]:
y.shape

(18285,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.14.0'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
messages.reset_index(inplace=True)

In [22]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
#corpus

In [29]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [30]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1294, 2503, 1223, 4615, 3235, 1623, 1370, 169, 3359, 1457],
 [4473, 725, 1230, 2564, 400, 3847, 4190],
 [3531, 2274, 2782, 292],
 [4002, 120, 4780, 1813, 2356, 2071],
 [1302, 400, 4852, 4615, 1401, 2971, 400, 4171, 2649, 1848],
 [914,
  138,
  3440,
  1948,
  2264,
  4283,
  1145,
  187,
  3030,
  4244,
  4421,
  4108,
  1701,
  2885,
  4190],
 [2907, 2270, 3647, 622, 3286, 4607, 2773, 3747, 79, 3071, 1898],
 [4601, 3677, 992, 4889, 3189, 4477, 4283, 69, 79, 3071, 1898],
 [1065, 1001, 2902, 4146, 4568, 1720, 297, 351, 4283, 1144],
 [4659, 3582, 2569, 2631, 548, 1116, 3753, 808],
 [2776, 2589, 3985, 637, 593, 3263, 1771, 2436, 832, 1602, 46],
 [1813, 4908, 3235, 1720, 4283, 3189],
 [3045, 950, 2974, 1574, 4349, 4271, 2491, 4666, 2190],
 [2544, 3579, 4125, 210, 723, 2606, 2247, 79, 3071, 1898],
 [1114, 3394, 607, 4062, 2320, 79, 3071, 1898],
 [3691, 3692, 2614, 524, 745, 2086, 1988, 1845, 2125, 83],
 [2010, 2920, 725],
 [3600, 1675, 1178, 2355, 4283, 385, 2912, 4190],
 [4306, 4844, 123

In [31]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [32]:
onehot_repr[1]

[4473, 725, 1230, 2564, 400, 3847, 4190]

### Embedding Representation

In [33]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1294 2503 1223 ...    0    0    0]
 [4473  725 1230 ...    0    0    0]
 [3531 2274 2782 ...    0    0    0]
 ...
 [ 734 2248 4026 ...    0    0    0]
 [1552 3189 2156 ...    0    0    0]
 [3119 4365 1477 ...    0    0    0]]


In [34]:
embedded_docs[1]

array([4473,  725, 1230, 2564,  400, 3847, 4190,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [35]:
embedded_docs[0]

array([1294, 2503, 1223, 4615, 3235, 1623, 1370,  169, 3359, 1457,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [36]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [37]:
len(embedded_docs),y.shape

(18285, (18285,))

In [38]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [39]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [49]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Epoch 1/5
192/192 [==============================] - 14s 55ms/step - loss: 0.3285 - accuracy: 0.8367 - val_loss: 0.2109 - val_accuracy: 0.9107
Epoch 2/5
192/192 [==============================] - 3s 14ms/step - loss: 0.1645 - accuracy: 0.9370 - val_loss: 0.2084 - val_accuracy: 0.9132
Epoch 3/5
192/192 [==============================] - 2s 13ms/step - loss: 0.1181 - accuracy: 0.9544 - val_loss: 0.2151 - val_accuracy: 0.9122
Epoch 4/5
192/192 [==============================] - 2s 10ms/step - loss: 0.0929 - accuracy: 0.9663 - val_loss: 0.2324 - val_accuracy: 0.9084
Epoch 5/5
192/192 [==============================] - 2s 12ms/step - loss: 0.0726 - accuracy: 0.9740 - val_loss: 0.2881 - val_accuracy: 0.9041


### Adding Dropout

In [56]:
#from tensorflow.keras.layers import Dropout
## Creating model
#embedding_vector_features=40
#model=Sequential()
#model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
#model.add(Dropout(0.3))
#model.add(LSTM(100))
#model.add(Dropout(0.3))
#model.add(Dense(1,activation='sigmoid'))
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [50]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [52]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [45]:
from sklearn.metrics import confusion_matrix

In [53]:
confusion_matrix(y_test,y_pred)

array([[3207,  212],
       [ 391, 2225]])

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9000828500414251

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      3419
           1       0.91      0.85      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.89      0.90      6035
weighted avg       0.90      0.90      0.90      6035

